## UKB Multivariate fine-mapping workflow

1. Sufficient statistics input XtX, XtY, YtY and n. We assume covariates C have been removed from X and Y. We provide a procedure to implement this.
2. GWAS summary statistics input z and R. We assume z scores have been computed after removal of covariates C.

In [ ]:
[global]
import glob
# single column file each line is the data filename
parameter: analysis_units = path
# Path to data directory
parameter: data_dir = path
# data file suffix
parameter: data_suffix = str
# Path to work directory where output locates
parameter: wd = path("./output")
# Path to prior data file: an RDS file with `U` and `w` for prior matrices and weights
parameter: prior = path('.')
# Path to residual cor/cov data file
parameter: resid_cor = path('.')
# Only analyze `cis` variants -- cis = N means using N variants around the center column of X matrix  
parameter: cis = 'NULL'
regions = [x.strip() for x in open(analysis_units).readlines() if x.strip() and not x.strip().startswith('#')]
genes = [f"{data_dir:a}/{x}.{data_suffix}" for x in regions if path(f"{data_dir:a}/{x}.{data_suffix}").exists()]

In [ ]:
# Convert LD store file to RDS format
[ldstore_to_rds]
# An identifier for your run of analysis
parameter: name = str
parameter: ld_dir = path
ld_files = glob.glob(f"{ld_dir:a}/{name}*.matrix")
input: ld_files, group_by = 1
output: f"{wd:a}/{_input:bn}.ld.rds"
task: trunk_workers = 1, trunk_size = 1, walltime = '12h', mem = '20G', cores = 2, tags = f'{step_name}_{_output:bn}'
R: expand = "${ }"
    ld = as.matrix(data.table::fread(${_input:r}))
    saveRDS(ld, ${_output:r})

In [ ]:
[sufficient_summary_stats_preprocessing]
parameter: phenoFile = path
parameter: covarFile = path
# path to z score file
parameter: z_dir = path()
parameter: z_suffix = str
# path to LD file
parameter: ld_dir = path()
parameter: ld_suffix = str
input: genes, group_by = 1
output: suffstats = f"{wd:a}/{_input:bn}.sufficient_stats.rds", 
        sumstats =  f"{wd:a}/{_input:bn}.summary_stats.rds"
task: trunk_workers = 1, trunk_size = 1, walltime = '4h', mem = '200G', cores = 1, tags = f'{step_name}_{_output[0]:bn}'
R: expand = '${ }', stdout = f"{_output[0]:nn}.stdout", stderr = f"{_output[0]:nn}.stderr"
    # FIXME: in practice we might need to 
    geno_file = ${_input:nr}
    z.file = "${z_dir:a}/${_input:bn}.${z_suffix}"
    ld.file = "${ld_dir:a}/${_input:bn}.${ld_suffix}"
    library(data.table)
    library(dplyr)
    
    X <- fread(paste0(geno_file, '.raw.gz'),sep = "\t",header = TRUE,stringsAsFactors = FALSE)
    map <- X[,1:6]
    X = X[, c('FID','IID','PAT','MAT','SEX', 'PHENOTYPE') := NULL]
    X <- as.matrix(X)
    
    X.info = fread(paste0(geno_file, '.pvar'),sep = "\t",header = TRUE,stringsAsFactors = FALSE)
    
    # Read phenotype data
    cat("Reading phenotype data.\n")
    pheno <- suppressMessages(fread(${phenoFile:r}))

    cat("Reading covariate file.\n")
    Z = suppressMessages(fread(${covarFile:r}))

    match.idx = match(map$IID, pheno$IID)
    pheno = pheno[match.idx,]
    match.idx = match(map$IID, Z$IID)
    Z = Z[match.idx,]
  
    Y = pheno %>% select(-FID, -IID) %>% as.matrix
    Z = Z %>% select(-FID, -IID) %>% as.matrix
  
    # centering
    Y = sweep(Y, 2, colMeans(Y), '-')
    Z = sweep(Z, 2, colMeans(Z), '-')
  
    A   <- crossprod(Z) # Z'Z
    # chol decomposition for (Z'Z)^(-1)
    R = chol(solve(A)) # R'R = (Z'Z)^(-1)
    W = R %*% crossprod(Z, X) # RZ'X
    S = R %*% crossprod(Z, Y) # RZ'Y

    SNPnames = colnames(X)
    rm(X)
    rm(Z)

    zscores = readRDS(z.file)

    # Load LD matrix from raw genotype
    ld = readRDS(ld.file)
    XtX = sqrt(zscores$XtXD) * t(ld*sqrt(zscores$XtXD)) - crossprod(W) # W'W = X'ZR'RZ'X = X'Z(Z'Z)^{-1}Z'X
    XtX = as.matrix(XtX)
    rownames(XtX) = colnames(XtX) = SNPnames
    R = cov2cor(XtX)

    # X'Y
    ## flip sign because X flip the REF, ALT
    XtY = -as.matrix(zscores$XtY - crossprod(W, S)) # W'S = X'ZR'RZ'y = X'Z(Z'Z)^{-1}Z'y

    # YtY
    YtY = as.matrix(crossprod(Y) - crossprod(S))

    Z = as.matrix(zscores$Z)
    rownames(Z) = SNPnames
    
    meta = zscores$pos[,1:5]
    if(!all.equal(meta, X.info, check.attributes = FALSE)){
        stop("ALLELE doesn't amtch.")
    }

    saveRDS(list(XtX = XtX, XtY = XtY, YtY = YtY, N = nrow(Y), meta = zscores$pos), ${_output["suffstats"]:r})
    saveRDS(list(Z = Z, LD = R, meta = zscores$pos, ld.file = ld.file), ${_output["sumstats"]:r})

In [ ]:
[univariate_analysis_1]
parameter: max_L = 10
input: genes, group_by = 1
output: suff = f"{wd:a}/{_input:bnn}.susiesuff.rds", 
        rss_rem_covariates =  f"{wd:a}/{_input:bnn}.susierss_rem_covariates.rds",
        rss_notrem_covariates =  f"{wd:a}/{_input:bnn}.susierss_notrem_covariates.rds"
task: trunk_workers = 1, trunk_size = 1, walltime = '5h', mem = '30G', cores = 1, tags = f'{step_name}_{_output[0]:bnn}'
R: expand = '${ }', stdout = f"{_output[0]:nn}.stdout", stderr = f"{_output[0]:nn}.stderr"
    library(susieR)
    dat_suff = readRDS('${_input:bnn}.sufficient_stats.rds')
    dat_rss = readRDS('${_input:bnn}.summary_stats.rds')
    R = readRDS(readRDS(dat_rss$ld.file))
    fitted_suff = list()
    fitted_rss_rem_covariates = list()
    fitted_rss_notrem_covariates = list()
    for (r in 1:ncol(dat_suff$XtY)) {
        ## sufficient stats
        st = proc.time()
        fitted_suff[[r]] <- susieR::susie_suff_stat(XtX = dat_suff$XtX, 
                                               Xty = dat_suff$Xty[,r]
                                               yty = dat_suff$YtY[r,r], n = dat_suff$N,
                                               L=${max_L},
                                               max_iter=1000,
                                               estimate_residual_variance=TRUE,
                                               estimate_prior_variance=TRUE,
                                               refine=TRUE)
        fitted_suff[[r]]$time = proc.time() - st
        fitted_suff[[r]]$cs_corr = susieR:::get_cs_correlation(fitted_suff[[r]], Xcorr=cov2cor(dat_suff$XtX))
        
        ## rss, LD correct for covariates
        st = proc.time()
        fitted_rss_rem_covariates[[r]] <- susieR::susie_rss(z = dat_rss$Z[,r],
                                                            R = dat_rss$LD,
                                                            L=${max_L},
                                                            max_iter=1000,
                                                            estimate_prior_variance=TRUE,
                                                            refine=TRUE)
        fitted_rss_rem_covariates[[r]]$time = proc.time() - st
        fitted_rss_rem_covariates[[r]]$cs_corr = susieR:::get_cs_correlation(fitted_rss_rem_covariates[[r]], 
                                                                             Xcorr=dat_rss$LD)
        
        ## rss, LD not correct for covariates
        st = proc.time()
        fitted_rss_notrem_covariates[[r]] <- susieR::susie_rss(z = dat_rss$Z[,r],R = R,
                                                               L=${max_L},max_iter=1000,
                                                               estimate_prior_variance=TRUE,
                                                               refine=TRUE)
        fitted_rss_notrem_covariates[[r]]$time = proc.time() - st
        fitted_rss_notrem_covariates[[r]]$cs_corr = susieR:::get_cs_correlation(fitted_rss_notrem_covariates[[r]], 
                                                                                Xcorr=R)
        
        names(fitted_suff) = names(fitted_rss_rem_covariates) = names(fitted_rss_notrem_covariates) = colnames(dat_suff$XtY)
        
    }
    
    saveRDS(fitted_suff, ${_output["suff"]:r})
    saveRDS(fitted_rss_rem_covariates, ${_output["rss_rem_covariates"]:r})
    saveRDS(fitted_rss_notrem_covariates, ${_output["rss_notrem_covariates"]:r})

In [ ]:
[sufficient_stats_analysis_1]
parameter: max_L = 10
input: genes, group_by = 1
output: f'{wd:a}/{_input:bnn}{resid_cor:bnx}.mvsusiesuff.rds'
task: trunk_workers = 1, trunk_size = 1, walltime = '2h', mem = '30G', cores = 1, tags = f'{step_name}_{_output:bn}'
R: expand = '${ }', stdout = f"{_output:n}.stdout", stderr = f"{_output:n}.stderr"
    get_prior_indices <- function(Z, U) {
      # make sure the prior col/rows match the colnames of the Y matrix
      z_names = colnames(Z)
      u_names = colnames(U)
      if (is.null(z_names) || is.null(u_names)) {
          return(NULL)
      } else if (identical(z_names, u_names)) {
          return(NULL)
      } else {
          return(match(z_names, u_names))
      }
    }

    library(mvsusieR)
    dat = readRDS(${_input:r})
    V = readRDS(${resid_cor:r})
    prior = readRDS(${prior:r})
    print(paste("Number of components in the mixture prior:", length(prior$U)))
    prior = mvsusieR::create_mash_prior(mixture_prior=list(weights=prior$w, matrices=prior$U), 
                                        include_indices = get_prior_indices(dat$XtY, prior$U[[1]]), 
                                        max_mixture_len=-1)
    st = proc.time()
    mv_res = mvsusieR::mvsusie_suff_stat(dat$XtX, dat$XtY, dat$YtY, dat$N, L=${max_L}, 
                                         prior_variance=prior, residual_variance=V, 
                                         precompute_covariances=T, compute_objective=T, 
                                         estimate_residual_variance=F, estimate_prior_variance=T, 
                                         estimate_prior_method='EM',max_iter = 1000, n_thread=1)
    mv_res$time = proc.time() - st
    if(mv_res$convergence$converged == FALSE){
        stop('Fail to converge.')
    }
    mv_res$cs_corr = susieR:::get_cs_correlation(mv_res, Xcorr=cov2cor(dat$XtX))
    saveRDS(mv_res, ${_output:r})

In [ ]:
[summary_stats_analysis_1]
parameter: max_L = 10
parameter: ld_type = 'original'
input: genes, group_by = 1
output: f'{wd:a}/{_input:bnn}.LD{ld_type}{resid_cor:bnx}.mvsusierss.rds'
task: trunk_workers = 1, trunk_size = 1, walltime = '2h', mem = '30G', cores = 1, tags = f'{step_name}_{_output:bn}'
R: expand = '${ }', stdout = f"{_output:n}.stdout", stderr = f"{_output:n}.stderr"
    get_prior_indices <- function(Z, U) {
      # make sure the prior col/rows match the colnames of the Y matrix
      z_names = colnames(Z)
      u_names = colnames(U)
      if (is.null(z_names) || is.null(u_names)) {
          return(NULL)
      } else if (identical(z_names, u_names)) {
          return(NULL)
      } else {
          return(match(z_names, u_names))
      }
    }

    library(mvsusieR)
    dat = readRDS(${_input:r})
    V = readRDS(${resid_cor:r})
    prior = readRDS(${prior:r})
    print(paste("Number of components in the mixture prior:", length(prior$U)))
    prior = mvsusieR::create_mash_prior(mixture_prior=list(weights=prior$w, matrices=prior$U), 
                                        include_indices = get_prior_indices(dat$Z, prior$U[[1]]), 
                                        max_mixture_len=-1)
    if("${ld_type}" == 'original'){
        R = readRDS(dat$ld.file)
    }else if("${ld_type}" == 'remove_cov'){
        R = dat$LD
    }
    st = proc.time()
    mv_res = mvsusieR::mvsusie_rss(dat$Z, R, L=${max_L}, 
                                   prior_variance=prior, residual_variance=V, 
                                   precompute_covariances=T, compute_objective=T, 
                                   estimate_prior_variance=T, estimate_prior_method='EM',
                                   max_iter = 1000, n_thread=1)
    mv_res$time = proc.time() - st
    if(mv_res$convergence$converged == FALSE){
        stop('Fail to converge.')
    }
    mv_res$cs_corr = susieR:::get_cs_correlation(mv_res, Xcorr=R)
    saveRDS(mv_res, ${_output:r})
            